#### (I). Inference on HumanEval: Phi3 & llama3-8B
* Pre-trained model requires less time to generate completion | They also have higher accuracy on HumanEval
* Instruction model requires mode time to generate response | Accuracy also laggs a bit more
* Error / Success with Message information included

In [1]:
from sample_hf import CodeCompletionGenerator
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm

model_name = "microsoft/Phi-3-mini-128k-instruct"
# model_name="meta-llama/Meta-Llama-3-8B"
is_instruct = True
file_name = "eval_phi_3_instruct"

code_gen = CodeCompletionGenerator(model_name=model_name, device="mps", is_instruct=is_instruct)

problems = read_problems()
# problems = {k: problems[k] for k in list(problems.keys())[:1]} # SubSample Problems out of the 163 cases

num_samples_per_task = 2
total_tasks = len(problems)
pb = tqdm(total=total_tasks * num_samples_per_task, desc="Generating completions")
samples = []
for task_id in problems:
    for _ in range(num_samples_per_task):
        completion = code_gen.generate_one_completion(problems[task_id]["prompt"])
        samples.append(dict(task_id=task_id, completion=completion))
        pb.update(1)
pb.close()
write_jsonl(f"{file_name}.jsonl", samples)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

[WARNING] rope_scaling 'type' currently only supports 'linear' setting rope scaling to false.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating completions:   9%|▉         | 29/328 [06:48<2:16:49, 27.46s/it]

KeyboardInterrupt: 

MLX generation approach is not right, the error rate is way to high ...

Try something else here

In [1]:
from sample_hf import check_performance
file_name="eval_phi_3_instruct"
check_performance(file_name)

Writing results to eval_phi_3_instruct.jsonl_results.jsonl...


100%|██████████| 328/328 [00:27<00:00, 11.87it/s]
